# Multi-Agent Research Assistant Using Meta LLaMA API (OpenRouter)

This notebook demonstrates how to build a **multi-agent AI system** using the **Meta LLaMA 3.3 model** accessed via **OpenRouter**.  
Previously, similar projects were showcased using the Gemini API. Here, we adapt the same architecture but with a free & publicly accessible model.

The system consists of three agents:
1. **Planner Agent** — Breaks down a high-level topic into smaller research tasks  
2. **Researcher Agent** — Executes those tasks and gathers insights  
3. **Summarizer Agent** — Combines insights into a final summary  

We’ll pass prompts and messages between these agents to accomplish end-to-end research (e.g. “Do research on Tesla stock in upcoming days”).

In [1]:
import time
import os
import requests

## 1. Setup & Configuration

- We use the **OpenRouter API endpoint** pointing to the model `meta-llama/llama-3.3-70b-instruct:free`.
- Replace `"llama_api_key"` below with your actual OpenRouter API key.
- The API endpoint is: `https://openrouter.ai/api/v1/chat/completions`
- We wrap API calls in a helper function `meta_generate(...)` that sends our prompt to the model, handles responses, and returns the generated text.

This approach is analogous to how the Gemini API was used in other tutorials, but with the LLaMA model under the hood which is free.

In [ ]:
API_KEY = "YOUR_API_KEY"  # <-- Replace this with your OpenRouter API key
MODEL_NAME = "meta-llama/llama-3.3-70b-instruct:free"
API_URL = "https://openrouter.ai/api/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [3]:
# -----------------------------
# Helper: generate text using OpenRouter Meta LLaMA model
# -----------------------------

def meta_generate(prompt: str, max_tokens: int = 512) -> str:
    """Send `prompt` to OpenRouter Meta LLaMA model and return generated text."""
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload, timeout=60)
        response.raise_for_status()
        data = response.json()

        # Extract content from response
        if "choices" in data and len(data["choices"]) > 0:
            return data["choices"][0]["message"]["content"].strip()
        else:
            return str(data)

    except Exception as e:
        return f"[OpenRouter API Error] {e}"

## 2. Agent Design & Prompting

Each agent has a distinct role and prompt structure:

### Planner Agent  
- Role: Breaks the main topic into sub-topics or research questions  
- Prompt style: “You are a planner agent … create a detailed step-by-step plan to research the topic: …”  
- Returns a textual plan.

### Researcher Agent  
- Role: Takes the plan from the planner and executes research — gathering facts, data, recent developments  
- Prompt style: “You are a researcher agent … Based on this plan … produce a concise report … with bullet points and highlights.”  
- Returns a research report.

### Summarizer Agent  
- Role: Synthesizes the research into a clean, final summary with key insights, risks, and a conclusion  
- Prompt style: “You are a summarizer agent … Summarize the following research results … into a short clear summary …”  
- Returns the final summary.

By defining the agent roles precisely in the prompts, we guide the LLaMA model to behave in a modular, structured way.

In [4]:
# Agents using meta_generate
# -----------------------------

def planner_agent(task: str) -> str:
    prompt = (
        "You are a planner agent. Create a detailed step-by-step plan to research the topic: "
        f"{task}\n\nInclude a short list of sources to check (news, SEC filings, earnings, analyst notes)."
    )
    return meta_generate(prompt, max_tokens=400)


def researcher_agent(plan: str) -> str:
    prompt = (
        "You are a researcher agent. Based on the following plan, gather insights and produce a concise report."
        f"\n\nPlan:\n{plan}\n\nProvide bullet points with findings, recent news highlights, and an actionable summary."
    )
    return meta_generate(prompt, max_tokens=800)


def summarizer_agent(results: str) -> str:
    prompt = (
        "You are a summarizer agent. Summarize the following research results into a short, clear summary "
        "with key takeaways, risks, and a one-paragraph conclusion."
        f"\n\nResults:\n{results}"
    )
    return meta_generate(prompt, max_tokens=300)


## 3. Multi-Agent Orchestration & Pipeline

The system works in this sequence:

1. Receive a **task** (e.g. “Research Tesla stock in the upcoming days”)  
2. Call **Planner Agent** → get a plan  
3. Pass the plan into **Researcher Agent** → get research findings  
4. Pass findings into **Summarizer Agent** → get final summary  
5. Print intermediate outputs and final result  

This chaining of agents allows us to decompose complex tasks, maintain modularity, and produce structured outputs.  
It mirrors concepts from multi-agent AI systems, where each agent specializes in one role.

In [5]:
# -----------------------------
# Multi-Agent Pipeline
# -----------------------------

def multi_agent_research(task: str):
    print("Hello! I am your AI Research Assistant.\n")
    print(f"Starting research process for: '{task}'\n")

    print("Planner Agent: Creating a research plan...\n")
    plan = planner_agent(task)
    print("🧩 Planner Agent Output:\n", plan, "\n")

    time.sleep(0.5)

    print("Researcher Agent: Executing the plan...\n")
    results = researcher_agent(plan)
    print("📊 Researcher Agent Output:\n", results, "\n")

    time.sleep(0.5)

    print("Summarizer Agent: Summarizing findings...\n")
    summary = summarizer_agent(results)
    print("🧾 Summarizer Agent Output:\n", summary, "\n")

    print("✅ Research process completed successfully!")


## 4. Example Run & Output

Here’s a sample flow:

- **Task**: `Do research on Tesla stock in the upcoming days`
- **Planner Agent Output**:  
  - “1. Check recent Tesla earnings; 2. Review latest news articles; 3. Analyze analyst consensus; 4. Evaluate risks and future outlook”
- **Researcher Agent Output**:  
  - Bullet points summarizing recent earnings, news, market sentiment, competitor outlooks, etc.
- **Summarizer Agent Output**:  
  - A concise paragraph capturing key takeaways, risks, and potential movement in Tesla stock.

You’ll see each stage printed above so viewers can track how the system works step by step.

In [7]:
# -----------------------------
# Run if executed as a script or cell
# -----------------------------
if __name__ == "__main__":
    task_text = "Do research on cheap flights available from India to Dubai in the next 3 months."
    multi_agent_research(task_text)

Hello! I am your AI Research Assistant.

Starting research process for: 'Do research on cheap flights available from India to Dubai in the next 3 months.'

Planner Agent: Creating a research plan...

🧩 Planner Agent Output:
 To research the topic of cheap flights available from India to Dubai in the next 3 months, I will follow this detailed step-by-step plan:

**Step 1: Define the Research Objective**
 Identify the specific research goal: finding cheap flights from India to Dubai for the next 3 months.

**Step 2: Determine the Relevant Sources**
 Compile a list of sources to check for information on cheap flights:
- Airline websites (e.g., Emirates, IndiGo, SpiceJet)
- Flight comparison websites (e.g., Skyscanner, Kayak, Google Flights)
- Travel booking websites (e.g., Expedia, Booking.com)
- News articles and blogs related to travel and aviation
- Social media and forums (e.g., Twitter, Facebook, Reddit)

**Step 3: Identify Key Search Parameters**
 Determine the key parameters to use